<a href="https://colab.research.google.com/github/Debopam-Chowdhury/VFL_Sem_8_project/blob/Soumi/VFL_New_Hopefully_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install
!pip install torch torchvision tqdm scikit-learn pandas numpy matplotlib --quiet

In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from PIL import Image
from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm
import os
import json
import warnings
warnings.filterwarnings('ignore')
import timm
from peft import get_peft_model, LoraConfig, TaskType


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
torch.manual_seed(42)
np.random.seed(42)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [5]:
import pandas as pd

In [6]:
print("Loading datasets...")
df_ratings = pd.read_csv('/content/drive/MyDrive/ratings.csv')
df_movies = pd.read_csv('/content/drive/MyDrive/movies_new.csv')
df_credits = pd.read_csv('/content/drive/MyDrive/credits.csv')
df_tags = pd.read_csv('/content/drive/MyDrive/tags.csv')
df_genome_scores = pd.read_csv('/content/drive/MyDrive/genome-scores.csv')
df_genome_tags = pd.read_csv('/content/drive/MyDrive/genome-tags.csv')

# Drop timestamp from ratings
if 'timestamp' in df_ratings.columns:
    df_ratings = df_ratings.drop('timestamp', axis=1)

# Merge genome data
print("Merging genome data...")
df_genomes = pd.merge(df_genome_scores, df_genome_tags, on='tagId', how='left')

Loading datasets...
Merging genome data...


In [7]:
print("Finding common movies across all datasets...")
common_movies = set(df_ratings['movieId'].unique())
common_movies = common_movies.intersection(set(df_movies['movieId'].unique()))
common_movies = common_movies.intersection(set(df_credits['id'].unique()))
common_movies = common_movies.intersection(set(df_genomes['movieId'].unique()))
common_movies = common_movies.intersection(set(df_tags['movieId'].unique()))
common_movies = sorted(list(common_movies))

print(f"Number of common movies across all datasets: {len(common_movies)}")

Finding common movies across all datasets...
Number of common movies across all datasets: 3024


In [8]:
df_ratings = df_ratings[df_ratings['movieId'].isin(common_movies)].reset_index(drop=True)
df_movies = df_movies[df_movies['movieId'].isin(common_movies)].reset_index(drop=True)
df_credits = df_credits[df_credits['id'].isin(common_movies)].reset_index(drop=True)
df_genomes = df_genomes[df_genomes['movieId'].isin(common_movies)].reset_index(drop=True)
df_tags = df_tags[df_tags['movieId'].isin(common_movies)].reset_index(drop=True)

In [9]:
print("Encoding users and movies...")
user_encoder = LabelEncoder()
movie_encoder = LabelEncoder()

df_ratings['user_idx'] = user_encoder.fit_transform(df_ratings['userId'])
df_ratings['movie_idx'] = movie_encoder.fit_transform(df_ratings['movieId'])

num_users = df_ratings['user_idx'].nunique()
num_movies = df_ratings['movie_idx'].nunique()

print(f"\nDataset Statistics:")
print(f"  Number of users: {num_users}")
print(f"  Number of movies: {num_movies}")
print(f"  Number of ratings: {len(df_ratings)}")
print(f"  Rating range: {df_ratings['rating'].min()} to {df_ratings['rating'].max()}")

Encoding users and movies...

Dataset Statistics:
  Number of users: 138493
  Number of movies: 3024
  Number of ratings: 9216170
  Rating range: 0.5 to 5.0


In [10]:
user_idx_tensor  = torch.tensor(df_ratings['user_idx'].values, dtype=torch.long).to(device)
movie_idx_tensor = torch.tensor(df_ratings['movie_idx'].values, dtype=torch.long).to(device)


In [11]:
posters_directory = "/content/drive/MyDrive/ml-20m-psm/posters"

In [12]:
# =====================================
# FIXED Aligned VFL Dataset for Opacus
# =====================================
class AlignedVFLDataset(Dataset):
    def __init__(self, df):
        self.data = df.reset_index(drop=True)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]

        user_idx  = torch.tensor(row["user_idx"], dtype=torch.long)
        movie_idx = torch.tensor(row["movie_idx"], dtype=torch.long)
        movieId   = torch.tensor(row["movieId"], dtype=torch.long)
        rating    = torch.tensor(row["rating"], dtype=torch.float32)

        # Combine 3 features into a single vector (Opacus-friendly)
        X = torch.stack([user_idx, movie_idx, movieId])  # [3]
        y = rating

        return X, y


In [13]:
print("\nCreating train/test split...")
train_df, test_df = train_test_split(df_ratings, test_size=0.2, random_state=42)

# Take only first 500 samples from training data
train_df = train_df.head(500).reset_index(drop=True)

# For test set, take proportionally fewer (e.g., 100 samples)
test_df = test_df.head(100).reset_index(drop=True)

train_dataset = AlignedVFLDataset(train_df)
test_dataset = AlignedVFLDataset(test_df)




train_loader = DataLoader(
    train_dataset,
    batch_size=8,
    shuffle=True,
    num_workers=0,   # 👈 set to 0 temporarily
)

test_loader = DataLoader(
    test_dataset,
    batch_size=8,
    shuffle=False,
    num_workers=0,   # 👈 0 here as well
)


print(f"Train samples: {len(train_dataset)}")
print(f"Test samples: {len(test_dataset)}")
print(f"Batch size: 8")


Creating train/test split...
Train samples: 500
Test samples: 100
Batch size: 8


In [14]:
# Proper LightGCN-style GNN for all VFL clients (pure PyTorch, DP-friendly)
# Drop-in replacement for GNNBottomModel and initialization blocks

import torch
import torch.nn as nn
import torch.nn.functional as F

# ------------------------------------------------------------
# 1) Build item-item adjacency from co-ratings (sparse, symmetric)
# ------------------------------------------------------------

def build_item_graph(num_items, user_idx, item_idx, num_users, device):
    """
    user_idx, item_idx: 1D LongTensors aligned (ratings rows)
    Returns: normalized sparse adjacency A_hat (num_items x num_items)
    """
    # Build user->items lists
    ui = [[] for _ in range(num_users)]
    for u, i in zip(user_idx.tolist(), item_idx.tolist()):
        ui[u].append(i)

    # Count co-occurrence edges
    rows, cols, vals = [], [], []
    for items in ui:
        if len(items) < 2:
            continue
        uniq = list(set(items))
        for a in uniq:
            for b in uniq:
                if a == b:
                    continue
                rows.append(a)
                cols.append(b)
                vals.append(1.0)

    if len(rows) == 0:
        # fallback to identity
        idx = torch.arange(num_items, device=device)
        indices = torch.stack([idx, idx])
        values = torch.ones(num_items, device=device)
        A = torch.sparse_coo_tensor(indices, values, (num_items, num_items))
        return normalize_adj(A).coalesce()

    indices = torch.tensor([rows, cols], dtype=torch.long, device=device)
    values = torch.tensor(vals, dtype=torch.float32, device=device)
    A = torch.sparse_coo_tensor(indices, values, (num_items, num_items)).coalesce()
    return normalize_adj(A)


def normalize_adj(A):
    """Symmetric normalization D^{-1/2} A D^{-1/2}"""
    deg = torch.sparse.sum(A, dim=1).to_dense() + 1e-8
    d_inv_sqrt = torch.pow(deg, -0.5)
    D_left = torch.sparse_coo_tensor(
        torch.stack([torch.arange(A.size(0)), torch.arange(A.size(0))]).to(A.device),
        d_inv_sqrt, A.shape
    )
    D_right = D_left
    return torch.sparse.mm(torch.sparse.mm(D_left, A), D_right).coalesce()

# ------------------------------------------------------------
# 2) Proper LightGCN
# ------------------------------------------------------------

class LightGCNBottomModel(nn.Module):
    """
    Multi-layer message passing over item graph
    H^{k+1} = A_hat H^{k}
    Final embedding = mean_k H^{k}
    """
    def __init__(self, num_items, embedding_dim, A_hat, num_layers=2):
        super().__init__()
        self.num_items = num_items
        self.embedding_dim = embedding_dim
        self.num_layers = num_layers
        self.A_hat = A_hat  # sparse normalized adjacency (frozen)

        self.embedding = nn.Embedding(num_items, embedding_dim)
        nn.init.xavier_uniform_(self.embedding.weight)

    def propagate(self):
        H = self.embedding.weight
        out = H
        for _ in range(self.num_layers):
            H = torch.sparse.mm(self.A_hat, H)
            out = out + H
        out = out / (self.num_layers + 1)
        return out

    def forward(self, item_ids):
        Z = self.propagate()  # [num_items, d]
        return Z[item_ids]

# ------------------------------------------------------------
# 3) Helper to create 4 clients with same graph
# ------------------------------------------------------------

def create_all_clients(num_items, user_idx, item_idx, num_users, device):
    A_hat = build_item_graph(num_items, user_idx, item_idx, num_users, device)

    client_movies       = LightGCNBottomModel(num_items, 32,  A_hat, num_layers=2)
    client_credits      = LightGCNBottomModel(num_items, 64,  A_hat, num_layers=2)
    client_tags_posters = LightGCNBottomModel(num_items, 128, A_hat, num_layers=2)
    client_genomes      = LightGCNBottomModel(num_items, 32,  A_hat, num_layers=2)

    return client_movies, client_credits, client_tags_posters, client_genomes


In [20]:
# ============ CLIENT A: MOVIES MODEL ============
class MoviesBottomModel(nn.Module):
    def __init__(self, num_movies,gnn_model,num_genres, embedding_dim=64):
        super().__init__()
        self.movie_to_genres = {}
        self.num_genres = num_genres
        self.embedding_dim = embedding_dim
        self.gnn = gnn_model
        self.graph_fusion = nn.Linear(embedding_dim + 32, embedding_dim)

        # Encoder: genres → embedding
        self.encoder = nn.Sequential(
            nn.Linear(num_genres, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, embedding_dim)
        )

    def setup_genres_mapping(self, df_movies):
        """Create mapping from movieId to genre vector"""
        df_movies = df_movies.copy()
        df_movies['genres_list'] = df_movies['genres'].apply(
            lambda x: [] if x == "(no genres listed)" else x.split("|")
        )

        mlb = MultiLabelBinarizer()
        genres_matrix = mlb.fit_transform(df_movies['genres_list'])

        for idx, row in df_movies.iterrows():
            movie_id = row['movieId']
            self.movie_to_genres[movie_id] = genres_matrix[idx]

        print(f"Movies client: Loaded {len(self.movie_to_genres)} movies with {self.num_genres} genres")

    def forward(self, movieIds):
        if movieIds.numel() == 0:
          return torch.zeros((0, self.embedding_dim), device=movieIds.device)
        batch_size = movieIds.size(0)
        genre_vectors = torch.zeros(batch_size, self.num_genres, device=movieIds.device)

        for i, movie_id in enumerate(movieIds.cpu().numpy()):
            if movie_id in self.movie_to_genres:
                genre_vectors[i] = torch.tensor(self.movie_to_genres[movie_id], dtype=torch.float32)

        genre_embedding = self.encoder(genre_vectors)

        gnn_embedding = self.gnn(movieIds)

        final_embedding = self.graph_fusion(
            torch.cat([genre_embedding, gnn_embedding], dim=1)
        )

        return final_embedding

In [22]:
# ============ CLIENT B: CREDITS MODEL ============
class CreditsBottomModel(nn.Module):
    def __init__(self, num_movies,gnn_model,embedding_dim=128):
        super().__init__()
        self.gnn = gnn_model
        self.graph_fusion = nn.Linear(embedding_dim + 64, embedding_dim)
        self.movie_to_credits = {}
        self.embedding_dim = embedding_dim
        self.feature_dim = None
        self.encoder = None

    def setup_credits_mapping(self, df_credits):
        """Preprocess credits data and create embeddings"""
        df_credits = df_credits.copy()

        def extract_names(cast_json, limit=5):
            try:
                cast_list = json.loads(cast_json.replace("'", '"'))
                return [person.get('name', '') for person in cast_list[:limit]]
            except:
                return []

        # Extract top cast names
        df_credits['cast_names'] = df_credits['cast'].apply(lambda x: extract_names(x, 5))
        df_credits['cast_text'] = df_credits['cast_names'].apply(lambda x: ' '.join(x))

        # TF-IDF vectorization
        vectorizer = TfidfVectorizer(max_features=200, stop_words='english')
        credits_features = vectorizer.fit_transform(df_credits['cast_text']).toarray()

        self.feature_dim = credits_features.shape[1]

        # Initialize encoder
        self.encoder = nn.Sequential(
            nn.Linear(self.feature_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, self.embedding_dim),
            nn.ReLU()
        )

        # Store features
        for idx, row in df_credits.iterrows():
            movie_id = row['id']
            self.movie_to_credits[movie_id] = credits_features[idx]

        print(f"Credits client: Loaded {len(self.movie_to_credits)} movies with {self.feature_dim} features")

    def forward(self, movieIds):
        if self.encoder is None:
            raise RuntimeError("Must call setup_credits_mapping first!")

        batch_size = movieIds.size(0)
        credit_vectors = torch.zeros(batch_size, self.feature_dim, device=movieIds.device)

        for i, movie_id in enumerate(movieIds.cpu().numpy()):
            if movie_id in self.movie_to_credits:
                credit_vectors[i] = torch.tensor(self.movie_to_credits[movie_id], dtype=torch.float32)

        credit_embedding = self.encoder(credit_vectors)
        gnn_embedding = self.gnn(movieIds)

        return self.graph_fusion(torch.cat([credit_embedding, gnn_embedding], dim=1))

In [17]:
!pip install open_clip_torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 5.2 MB/s eta 0:00:00


In [18]:
class LoRALinear(nn.Module):
    def __init__(self, base_layer, r=4, alpha=8):
        super().__init__()
        self.base = base_layer
        self.r = r
        self.lora_A = nn.Linear(base_layer.in_features, r, bias=False)
        self.lora_B = nn.Linear(r, base_layer.out_features, bias=False)
        self.scaling = alpha / r

        # init LoRA parameters small, freeze base
        nn.init.kaiming_uniform_(self.lora_A.weight, a=math.sqrt(5))
        nn.init.zeros_(self.lora_B.weight)
        for p in self.base.parameters():
            p.requires_grad = False

    def forward(self, x):
        return self.base(x) + self.scaling * self.lora_B(self.lora_A(x))


In [23]:
import open_clip
import torch
import torch.nn as nn
from PIL import Image
import os

# ============ CLIENT C: TAGS + POSTERS MODEL (Multimodal via CLIP) ============
class TagsPosterBottomModel(nn.Module):
    def __init__(self, num_movies,gnn_model,embedding_dim=512, device="cpu"):
        super().__init__()
        self.gnn = gnn_model
        self.final_fusion = nn.Linear(embedding_dim + 128, embedding_dim)
        self.movie_to_tags_text = {}   # raw combined tag text per movie
        self.movie_to_poster = {}
        self.embedding_dim = embedding_dim
        self.device = torch.device(device)

        # --- Load CLIP (open_clip) ---
        self.clip_model, self.clip_train_preprocess, self.clip_eval_preprocess = \
            open_clip.create_model_and_transforms(
                "ViT-B-32", pretrained="laion2b_e16"
            )
        self.clip_tokenizer = open_clip.get_tokenizer("ViT-B-32")
        self.clip_model = self.clip_model.to(self.device)
        self.clip_model.eval()  # keep backbone frozen

        # Dim of CLIP embeddings (image & text share this)
        self.clip_embed_dim = self.clip_model.text_projection.shape[1]

        # Fusion MLP over [img_emb ; txt_emb] -> embedding_dim
        self.fusion_mlp = nn.Sequential(
            nn.Linear(self.clip_embed_dim * 2, embedding_dim),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(embedding_dim, embedding_dim),
            nn.ReLU(),
        )

        self.has_posters = False

    def setup_tags_posters_mapping(self, df_tags, posters_dir=None):
        """Preprocess tags and posters data: store raw tag text + poster paths."""
        df_tags = df_tags.copy()
        print("Setting up Tags + Posters client (CLIP-based)...")

        # Combine tags to a single string per movie
        df_tags["tag"] = df_tags["tag"].astype(str)
        movie_tags = df_tags.groupby("movieId")["tag"].apply(
            lambda x: " ".join(x)
        ).reset_index()

        # 2) Image transform (CRITICAL FOR DP)
        # ----------------------------------------
        transform = transforms.Compose([
            transforms.Resize((224, 224)),   # fixed size required
            transforms.ToTensor(),
        ])


        # Posters
        poster_map = {}
        if posters_dir is not None and os.path.exists(posters_dir):
            self.has_posters = True
            for f in os.listdir(posters_dir):
                try:
                    mid = int(f.split(".")[0])
                    poster_map[mid] = os.path.join(posters_dir, f)
                except Exception:
                    continue
            print(f"Found {len(poster_map)} poster images")
        else:
            print("Warning: No posters directory. Using tags only.")
            self.has_posters = False

        # If we have posters, restrict to movies with both tags and posters
        if self.has_posters:
            movie_tags_filtered = movie_tags[movie_tags["movieId"].isin(poster_map.keys())].reset_index(drop=True)
            print(f"Movies with both tags and posters: {len(movie_tags_filtered)}")

            for _, row in movie_tags_filtered.iterrows():
                mid = int(row["movieId"])
                self.movie_to_tags_text[mid] = row["tag"]
                self.movie_to_poster[mid] = poster_map[mid]
        else:
            for _, row in movie_tags.iterrows():
                mid = int(row["movieId"])
                self.movie_to_tags_text[mid] = row["tag"]

        print(f"Tags+Posters client (CLIP): Loaded {len(self.movie_to_tags_text)} movies")

    def _load_and_process_image(self, poster_path):
        """Use CLIP's own preprocess pipeline for images, with safe fallback."""
        try:
            if not poster_path or not os.path.exists(poster_path):
                raise FileNotFoundError
            img = Image.open(poster_path).convert("RGB")
            img_tensor = self.clip_eval_preprocess(img)  # open_clip transform
            return img_tensor
        except Exception:
            # Fallback: CLIP expects 3x224x224-like tensor, use zeros
            return torch.zeros(3, 224, 224)

    def forward(self, movieIds):
        """
        movieIds: LongTensor [B]
        Returns: fused embedding [B, embedding_dim]
        """
        if movieIds is None or movieIds.numel() == 0:
            return torch.zeros((0, self.embedding_dim), device=self.device)
        batch_size = movieIds.size(0)
        device = movieIds.device

        # Ensure CLIP model is on the correct device
        if next(self.clip_model.parameters()).device != device:
            self.clip_model = self.clip_model.to(device)

        images = []
        tag_texts = []

        # Build per-movie text and image
        for mid in movieIds.cpu().numpy():
            mid = int(mid)

            # Text: tags string
            if mid in self.movie_to_tags_text:
                tag_texts.append(self.movie_to_tags_text[mid])
            else:
                tag_texts.append("")

            # Image: poster or zero-image
            if self.has_posters and mid in self.movie_to_poster:
                img_tensor = self._load_and_process_image(self.movie_to_poster[mid])
            else:
                img_tensor = torch.zeros(3, 224, 224)
            images.append(img_tensor)

        # Always non-empty because we append one image per movieId
        image_batch = torch.stack(images).to(device)          # [B, 3, 224, 224]
        text_tokens = self.clip_tokenizer(tag_texts).to(device)

        with torch.no_grad():  # CLIP backbone frozen
            img_emb = self.clip_model.encode_image(image_batch)   # [B, d]
            txt_emb = self.clip_model.encode_text(text_tokens)    # [B, d]

        # Normalize (standard CLIP practice)
        img_emb = img_emb / img_emb.norm(dim=-1, keepdim=True)
        txt_emb = txt_emb / txt_emb.norm(dim=-1, keepdim=True)

        # Fuse: concat + small MLP to embedding_dim
        fused_input = torch.cat([img_emb, txt_emb], dim=1)  # [B, 2d]
        clip_embedding = self.fusion_mlp(fused_input)

        gnn_embedding = self.gnn(movieIds)

        final_embedding = self.final_fusion(
            torch.cat([clip_embedding, gnn_embedding], dim=1)
        )

        return final_embedding            # [B, embedding_dim]




In [24]:
class GenomesBottomModel(nn.Module):
    def __init__(self, num_movies, num_tags,gnn_model, embedding_dim=64):
        super().__init__()
        self.gnn = gnn_model
        self.graph_fusion = nn.Linear(embedding_dim + 32, embedding_dim)
        self.movie_to_genome = {}
        self.num_tags = num_tags
        self.embedding_dim = embedding_dim

        self.encoder = nn.Sequential(
            nn.Linear(num_tags, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, embedding_dim)
        )

    def setup_genomes_mapping(self, df_genomes):
        """Create genome relevance vectors"""
        genome_pivot = df_genomes.pivot_table(
            index='movieId',
            columns='tagId',
            values='relevance',
            fill_value=0
        )

        for movie_id in genome_pivot.index:
            self.movie_to_genome[movie_id] = genome_pivot.loc[movie_id].values

        print(f"Genomes client: Loaded {len(self.movie_to_genome)} movies with {self.num_tags} genome tags")

    def forward(self, movieIds):
        batch_size = movieIds.size(0)
        genome_vectors = torch.zeros(batch_size, self.num_tags, device=movieIds.device)

        for i, movie_id in enumerate(movieIds.cpu().numpy()):
            if movie_id in self.movie_to_genome:
                genome_vectors[i] = torch.tensor(self.movie_to_genome[movie_id], dtype=torch.float32)

        genome_embedding = self.encoder(genome_vectors)
        gnn_embedding = self.gnn(movieIds)

        return self.graph_fusion(torch.cat([genome_embedding, gnn_embedding], dim=1))

In [25]:
class TopAggregatorModel(nn.Module):
    def __init__(self, num_users, total_embedding_dim, hidden_layers=[256, 128, 64]):
        super().__init__()

        # User embedding
        self.user_embedding = nn.Embedding(num_users, 50)

        # MLP layers
        layers = []
        input_dim = 50 + total_embedding_dim
        for hidden_dim in hidden_layers:
          layers.extend([
                  nn.Linear(input_dim, hidden_dim),
                  nn.ReLU(),
                  nn.GroupNorm(1, hidden_dim),  # 1 group = LayerNorm-like, DP-safe
                  nn.Dropout(0.3),
              ])
          input_dim = hidden_dim
        layers.append(nn.Linear(input_dim, 1))
        self.mlp = nn.Sequential(*layers)


    def forward(self, user_idx, combined_embeddings):
        user_emb = self.user_embedding(user_idx)
        fusion_input = torch.cat([user_emb, combined_embeddings], dim=1)
        output = self.mlp(fusion_input)
        return output.squeeze()

In [26]:
class BottomClient:
    def __init__(self, name, model, device):
        self.name = name
        self.model = model.to(device)
        self.device = device
        self.optimizer = optim.Adam(self.model.parameters(), lr=0.001)
        self.current_embeddings = None

    def forward(self, movieIds):
        """Forward pass through bottom model"""
        self.model.train()
        movieIds = movieIds.to(self.device)
        embeddings = self.model(movieIds)
        self.current_embeddings = embeddings
        return embeddings

    def backward(self, gradients):
        """Backward pass through bottom model"""
        if self.current_embeddings is None:
            raise RuntimeError("Must call forward() before backward()")

        self.optimizer.zero_grad()
        self.current_embeddings.backward(gradients)
        self.optimizer.step()

    def eval_mode(self):
        self.model.eval()

    def train_mode(self):
        self.model.train()

In [27]:
!pip install opacus


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.4/254.4 kB 9.5 MB/s eta 0:00:00


In [28]:
from opacus import PrivacyEngine

class VFLCoordinator:
    def __init__(self, bottom_clients, top_model, device):
        self.clients = bottom_clients
        self.top_model = top_model.to(device)
        self.device = device

        # Top model optimizer
        self.optimizer_top = optim.Adam(self.top_model.parameters(), lr=0.001)
        self.criterion = nn.MSELoss()

        # Will be set later with make_private()
        self.privacy_engine = None

    def make_private(self, train_loader, target_epsilon, target_delta, epochs, max_grad_norm=1.0):
      """
      Make top_model DP with epsilon-targeted training.

      target_epsilon: desired total ε after `epochs` epochs
      target_delta:   e.g. 1e-5
      epochs:         how many epochs you plan to train
      """
      self.privacy_engine = PrivacyEngine()
      self.top_model, self.optimizer_top, self.dp_train_loader = \
          self.privacy_engine.make_private_with_epsilon(
              module=self.top_model,
              optimizer=self.optimizer_top,
              data_loader=train_loader,
              target_epsilon=target_epsilon,
              target_delta=target_delta,
              epochs=epochs,
              max_grad_norm=max_grad_norm,
          )


    def forward_pass(self, movieIds, user_idx):
        """Collect embeddings from all clients and forward through top model"""
        embeddings_dict = {}
        embeddings_list = []

        for name, client in self.clients.items():
            emb = client.forward(movieIds)
            embeddings_dict[name] = emb
            embeddings_list.append(emb)

        combined_embeddings = torch.cat(embeddings_list, dim=1)
        combined_embeddings.retain_grad()
        predictions = self.top_model(user_idx, combined_embeddings)
        return predictions, embeddings_dict, combined_embeddings

    def backward_pass(self, loss, embeddings_dict, combined_embeddings):
        """Compute gradients and send back to clients"""
        self.optimizer_top.zero_grad()
        loss.backward(retain_graph=True)
        self.optimizer_top.step()

        # Split gradients for each client
        start_idx = 0
        for name, client in self.clients.items():
            emb = embeddings_dict[name]
            end_idx = start_idx + emb.size(1)

            grad_slice = combined_embeddings.grad[:, start_idx:end_idx]
            client.backward(grad_slice)

            start_idx = end_idx

    def train_epoch(self, data_loader):
      self.top_model.train()
      for client in self.clients.values():
          client.train_mode()

      total_loss = 0.0
      num_batches = 0

      for X, y in tqdm(data_loader, desc="Training"):
        X = X.to(self.device)
        y = y.to(self.device)

        user_idx = X[:, 0].long()
        movieIds = X[:, 1].long()
        ratings = y.float()


        predictions, embeddings_dict, combined_embeddings = self.forward_pass(
              movieIds, user_idx
          )
        loss = self.criterion(predictions, ratings)
        self.backward_pass(loss, embeddings_dict, combined_embeddings)

        total_loss += loss.item()
        num_batches += 1

      # Optional: report epsilon after each epoch (only if privacy_engine exists)
      if getattr(self, "privacy_engine", None) is not None:
          # New Opacus API
          epsilon = self.privacy_engine.get_epsilon(delta=1e-5)
          print(f"(DP) ε = {epsilon:.2f}, δ = 1e-5")

      return total_loss / num_batches


    def evaluate(self, data_loader):
        """Evaluate on test set (no noise)"""
        self.top_model.eval()
        for client in self.clients.values():
            client.eval_mode()

        total_loss = 0.0
        num_batches = 0

        with torch.no_grad():
          for X, y in tqdm(data_loader, desc="Evaluating"):
            X = X.to(self.device)      # [B, 3]
            y = y.to(self.device)
            if X.numel() == 0:
              continue

            user_idx = X[:, 0].long()
            movieIds = X[:, 1].long()
            ratings  = y.float()

            # collect embeddings from all clients
            embeddings_list = []
            for client in self.clients.values():
                emb = client.model(movieIds.to(self.device))
                embeddings_list.append(emb)

            combined_embeddings = torch.cat(embeddings_list, dim=1)
            predictions = self.top_model(user_idx, combined_embeddings)

            loss = self.criterion(predictions, ratings)
            total_loss += loss.item()
            num_batches += 1


        return total_loss / num_batches


In [ ]:
print("\n" + "="*60)
print("INITIALIZING VERTICAL FEDERATED LEARNING MODELS (MULTIMODAL + GNN)")
print("="*60)

# ------------------------------------------------------------
# Build tensors for graph
# ------------------------------------------------------------
print("\nPreparing interaction graph...")

user_idx_tensor  = torch.tensor(df_ratings['user_idx'].values, dtype=torch.long).to(device)
movie_idx_tensor = torch.tensor(df_ratings['movie_idx'].values, dtype=torch.long).to(device)

print(f"Users:  {num_users}")
print(f"Movies: {num_movies}")
print(f"Interactions: {len(user_idx_tensor)}")

# ------------------------------------------------------------
# Create graph encoders (NOT clients)
# ------------------------------------------------------------
gnn_movies, gnn_credits, gnn_tags, gnn_genomes = create_all_clients(
    num_items=num_movies,
    user_idx=user_idx_tensor,
    item_idx=movie_idx_tensor,
    num_users=num_users,
    device=device
)

# ------------------------------------------------------------
# Create REAL bottom clients (feature + graph fusion)
# ------------------------------------------------------------
print("\nInitializing Multimodal Clients...")

client_movies = MoviesBottomModel(num_movies, num_genres, gnn_movies, 64)
client_movies.setup_genres_mapping(df_movies)

client_credits = CreditsBottomModel(num_movies, gnn_credits, 128)
client_credits.setup_credits_mapping(df_credits)

client_tags_posters = TagsPosterBottomModel(num_movies, gnn_tags, 512, device=device)
client_tags_posters.setup_tags_posters_mapping(df_tags, posters_directory)

client_genomes = GenomesBottomModel(num_movies, num_genome_tags, gnn_genomes, 64)
client_genomes.setup_genomes_mapping(df_genomes)

# ------------------------------------------------------------
# Wrap into VFL clients
# ------------------------------------------------------------
clients_dict = {
    'movies': BottomClient('movies', client_movies, device),
    'credits': BottomClient('credits', client_credits, device),
    'tags_posters': BottomClient('tags_posters', client_tags_posters, device),
    'genomes': BottomClient('genomes', client_genomes, device)
}

# ------------------------------------------------------------
# Embedding dimension restored (multimodal)
# ------------------------------------------------------------
total_embedding_dim = 64 + 128 + 512 + 64

print(f"\nEmbedding Dimensions:")
print(f"  Movies:        64")
print(f"  Credits:      128")
print(f"  Tags+Posters: 512")
print(f"  Genomes:       64")
print(f"  {'─'*25}")
print(f"  Total:        {total_embedding_dim}")

# ------------------------------------------------------------
# Top model
# ------------------------------------------------------------
top_model = TopAggregatorModel(
    num_users=num_users,
    total_embedding_dim=total_embedding_dim,
    hidden_layers=[256, 128, 64]
)

print("\nMODEL INITIALIZATION COMPLETE")


INITIALIZING VERTICAL FEDERATED LEARNING MODELS (MULTIMODAL + GNN)

Preparing interaction graph...
Users:  138493
Movies: 3024
Interactions: 9216170


In [ ]:
import numpy as np

epsilon_targets = [0, 0.5, 1, 1.5, 2, 5, 10]
TARGET_DELTA = 1e-5
num_epochs = 20
all_results = []

for TARGET_EPSILON in epsilon_targets:

    print("\n" + "="*80)
    print(f"STARTING NEW EXPERIMENT FOR ε = {TARGET_EPSILON}")
    print("="*80)

    # ------------------------------------------------------------
    # 1) Build interaction graph (GNN)
    # ------------------------------------------------------------
    user_idx_tensor  = torch.tensor(df_ratings['user_idx'].values, dtype=torch.long).to(device)
    movie_idx_tensor = torch.tensor(df_ratings['movie_idx'].values, dtype=torch.long).to(device)

    gnn_movies, gnn_credits, gnn_tags, gnn_genomes = create_all_clients(
        num_items=num_movies,
        user_idx=user_idx_tensor,
        item_idx=movie_idx_tensor,
        num_users=num_users,
        device=device
    )

    # ------------------------------------------------------------
    # 2) Build FEATURE CLIENTS (restore modalities)
    # ------------------------------------------------------------
    client_movies = MoviesBottomModel(num_movies, num_genres, embedding_dim=64)
    client_movies.setup_genres_mapping(df_movies)

    client_credits = CreditsBottomModel(num_movies, embedding_dim=128)
    client_credits.setup_credits_mapping(df_credits)

    # --- POSTERS + TAGS + GNN ---
    client_tags_posters = TagsPosterBottomModel(
        num_movies=num_movies,
        gnn_model=gnn_tags,
        embedding_dim=512,
        device=device
    )
    client_tags_posters.setup_tags_posters_mapping(df_tags, posters_directory)

    client_genomes = GenomesBottomModel(num_movies, num_genome_tags, embedding_dim=64)
    client_genomes.setup_genomes_mapping(df_genomes)

    # ------------------------------------------------------------
    # 3) Wrap into VFL clients
    # ------------------------------------------------------------
    clients_dict = {
        'movies': BottomClient('movies', client_movies, device),
        'credits': BottomClient('credits', client_credits, device),
        'tags_posters': BottomClient('tags_posters', client_tags_posters, device),
        'genomes': BottomClient('genomes', client_genomes, device),
    }

    # ------------------------------------------------------------
    # 4) Rebuild top model (768 dimension restored)
    # ------------------------------------------------------------
    total_embedding_dim = 64 + 128 + 512 + 64

    top_model = TopAggregatorModel(
        num_users=num_users,
        total_embedding_dim=total_embedding_dim,
        hidden_layers=[256, 128, 64],
    )

    coordinator = VFLCoordinator(
        bottom_clients=clients_dict,
        top_model=top_model,
        device=device,
    )

    coordinator.top_model.train()

    # ------------------------------------------------------------
    # 5) Apply Differential Privacy
    # ------------------------------------------------------------
    if TARGET_EPSILON > 0:
        coordinator.make_private(
            train_loader=train_loader,
            target_epsilon=float(TARGET_EPSILON),
            target_delta=TARGET_DELTA,
            epochs=num_epochs,
            max_grad_norm=1.0,
        )
        train_loader_used = coordinator.dp_train_loader
    else:
        print("Running NON-DP baseline")
        train_loader_used = train_loader

    # ------------------------------------------------------------
    # 6) Training loop
    # ------------------------------------------------------------
    best_test_loss = float('inf')
    train_losses = []
    test_losses = []
    epsilons = []

    for epoch in range(num_epochs):

        print(f"\nEpoch {epoch+1}/{num_epochs}")

        train_loss = coordinator.train_epoch(train_loader_used)
        train_losses.append(train_loss)

        if TARGET_EPSILON > 0:
            eps = coordinator.privacy_engine.get_epsilon(delta=TARGET_DELTA)
        else:
            eps = 0

        epsilons.append(eps)

        test_loss = coordinator.evaluate(test_loader)
        test_losses.append(test_loss)

        best_test_loss = min(best_test_loss, test_loss)

        print(f"Train Loss: {train_loss:.4f} | Test Loss: {test_loss:.4f} | ε={eps:.3f}")

    # ------------------------------------------------------------
    # 7) Save results
    # ------------------------------------------------------------
    all_results.append({
        "target_eps": TARGET_EPSILON,
        "final_eps": epsilons[-1],
        "best_test_loss": best_test_loss,
        "train_losses": train_losses,
        "test_losses": test_losses,
        "epsilons": epsilons,
    })

print("\nAll experiments completed successfully.")

In [ ]:
import numpy as np

epsilon_targets = [0, 0.5, 1, 1.5, 2, 5, 10]
TARGET_DELTA = 1e-5
num_epochs = 20
all_results = []

for TARGET_EPSILON in epsilon_targets:

    print("\n" + "="*80)
    print(f"STARTING NEW EXPERIMENT FOR ε = {TARGET_EPSILON}")
    print("="*80)

    # ------------------------------------------------------------
    # 1) Build interaction graph (shared GNN encoders)
    # ------------------------------------------------------------
    user_idx_tensor  = torch.tensor(df_ratings['user_idx'].values, dtype=torch.long).to(device)
    movie_idx_tensor = torch.tensor(df_ratings['movie_idx'].values, dtype=torch.long).to(device)

    gnn_movies, gnn_credits, gnn_tags, gnn_genomes = create_all_clients(
        num_items=num_movies,
        user_idx=user_idx_tensor,
        item_idx=movie_idx_tensor,
        num_users=num_users,
        device=device
    )

    # ------------------------------------------------------------
    # 2) Build FEATURE + GRAPH clients
    # ------------------------------------------------------------
    client_movies = MoviesBottomModel(num_movies, num_genres, gnn_movies, 64)
    client_movies.setup_genres_mapping(df_movies)

    client_credits = CreditsBottomModel(num_movies, gnn_credits, 128)
    client_credits.setup_credits_mapping(df_credits)

    client_tags_posters = TagsPosterBottomModel(
        num_movies=num_movies,
        gnn_model=gnn_tags,
        embedding_dim=512,
        device=device
    )
    client_tags_posters.setup_tags_posters_mapping(df_tags, posters_directory)

    client_genomes = GenomesBottomModel(num_movies, num_genome_tags, gnn_genomes, 64)
    client_genomes.setup_genomes_mapping(df_genomes)

    # ------------------------------------------------------------
    # 3) Wrap into VFL clients
    # ------------------------------------------------------------
    clients_dict = {
        'movies': BottomClient('movies', client_movies, device),
        'credits': BottomClient('credits', client_credits, device),
        'tags_posters': BottomClient('tags_posters', client_tags_posters, device),
        'genomes': BottomClient('genomes', client_genomes, device),
    }

    # ------------------------------------------------------------
    # 4) Top model (multimodal dimension)
    # ------------------------------------------------------------
    total_embedding_dim = 64 + 128 + 512 + 64  # 768

    top_model = TopAggregatorModel(
        num_users=num_users,
        total_embedding_dim=total_embedding_dim,
        hidden_layers=[256, 128, 64],
    )

    coordinator = VFLCoordinator(
        bottom_clients=clients_dict,
        top_model=top_model,
        device=device,
    )

    coordinator.top_model.train()

    # ------------------------------------------------------------
    # 5) Apply Differential Privacy
    # ------------------------------------------------------------
    if TARGET_EPSILON > 0:
        coordinator.make_private(
            train_loader=train_loader,
            target_epsilon=float(TARGET_EPSILON),
            target_delta=TARGET_DELTA,
            epochs=num_epochs,
            max_grad_norm=1.0,
        )
        train_loader_used = coordinator.dp_train_loader
    else:
        print("Running NON-DP baseline")
        train_loader_used = train_loader

    # ------------------------------------------------------------
    # 6) Training loop
    # ------------------------------------------------------------
    best_test_loss = float('inf')
    train_losses = []
    test_losses = []
    epsilons = []

    for epoch in range(num_epochs):

        print(f"\nEpoch {epoch+1}/{num_epochs}")

        train_loss = coordinator.train_epoch(train_loader_used)
        train_losses.append(train_loss)

        if TARGET_EPSILON > 0:
            eps = coordinator.privacy_engine.get_epsilon(delta=TARGET_DELTA)
        else:
            eps = 0.0

        epsilons.append(eps)

        test_loss = coordinator.evaluate(test_loader)
        test_losses.append(test_loss)

        best_test_loss = min(best_test_loss, test_loss)

        print(f"Train Loss: {train_loss:.4f} | Test Loss: {test_loss:.4f} | ε={eps:.3f}")

    # ------------------------------------------------------------
    # 7) Save results
    # ------------------------------------------------------------
    all_results.append({
        "target_eps": TARGET_EPSILON,
        "final_eps": epsilons[-1],
        "best_test_loss": best_test_loss,
        "train_losses": train_losses,
        "test_losses": test_losses,
        "epsilons": epsilons,
    })

print("\nAll experiments completed successfully.")

In [ ]:
def predict_rating(coordinator, userId, movieId):
    """Predict rating for a specific user-movie pair"""
    coordinator.top_model.eval()
    for client in coordinator.clients.values():
        client.eval_mode()

    try:
        user_idx = user_encoder.transform([userId])[0]
    except:
        print(f"Error: User {userId} not in training set")
        return None

    # Prepare tensors
    user_idx_tensor = torch.tensor([user_idx], dtype=torch.long).to(device)
    movieId_tensor = torch.tensor([movieId], dtype=torch.long).to(device)

    with torch.no_grad():
        embeddings_list = []
        for client in coordinator.clients.values():
            emb = client.model(movieId_tensor)
            embeddings_list.append(emb)

        combined_embeddings = torch.cat(embeddings_list, dim=1)
        prediction = coordinator.top_model(user_idx_tensor, combined_embeddings)

    return prediction.item()


# Test on sample predictions
print("\n" + "="*70)
print(" "*20 + "SAMPLE PREDICTIONS")
print("="*70 + "\n")

# Get 5 random samples from test set
sample_indices = np.random.choice(len(test_dataset), size=5, replace=False)

for i, idx in enumerate(sample_indices, 1):
    sample = test_dataset[idx]

    userId = user_encoder.inverse_transform([sample['user_idx'].item()])[0]
    movieId = sample['movieId'].item()
    actual_rating = sample['rating'].item()

    predicted_rating = predict_rating(coordinator, userId, movieId)

    if predicted_rating is not None:
        error = abs(actual_rating - predicted_rating)

        print(f"Prediction {i}:")
        print(f"  User ID:          {userId}")
        print(f"  Movie ID:         {movieId}")
        print(f"  Actual Rating:    {actual_rating:.2f}")
        print(f"  Predicted Rating: {predicted_rating:.2f}")
        print(f"  Absolute Error:   {error:.2f}")
        print()

print("="*70)

# Calculate overall test RMSE
print("\nCalculating overall test RMSE...")
all_predictions = []
all_actuals = []

coordinator.top_model.eval()
for client in coordinator.clients.values():
    client.eval_mode()

with torch.no_grad():
    for batch in tqdm(test_loader, desc="Predicting"):
        user_idx = batch['user_idx'].to(device)
        movieIds = batch['movieId'].to(device)
        ratings = batch['rating'].to(device)

        embeddings_list = []
        for client in coordinator.clients.values():
            emb = client.model(movieIds)
            embeddings_list.append(emb)

        combined_embeddings = torch.cat(embeddings_list, dim=1)
        predictions = coordinator.top_model(user_idx, combined_embeddings)

        all_predictions.extend(predictions.cpu().numpy())
        all_actuals.extend(ratings.cpu().numpy())

all_predictions = np.array(all_predictions)
all_actuals = np.array(all_actuals)

mse = np.mean((all_predictions - all_actuals) ** 2)
rmse = np.sqrt(mse)
mae = np.mean(np.abs(all_predictions - all_actuals))

print(f"\n{'='*70}")
print(" "*25 + "FINAL METRICS")
print(f"{'='*70}")
print(f"  Test MSE:  {mse:.4f}")
print(f"  Test RMSE: {rmse:.4f}")
print(f"  Test MAE:  {mae:.4f}")
print(f"{'='*70}\n")


In [ ]:
### graph neural network pytorch geometry DGL
# 1. gat(graph attention neural network) based novel fusion mechanism -  primary client
# 2. GNN in each client
# 3. Differential Privacy for vertical federated learning(Opacus package)
# 4. Image embedding with Light weight VLM with LORA
# 5. upscale the data used
# 6. composite key (user_id, Movie_id)
# 7. searching for suitable reccomender system dataset(multimodal)